<a href="https://colab.research.google.com/github/Fernigithub/Agreed/blob/main/Agreed_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Installs geemap package
import subprocess

try:
        import geemap 
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
        print('geopandas package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
        print('tsmoothie package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'tsmoothie'])
        print('plotly.express package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'plotly.express'])
        subprocess.check_call(["python", '-m', 'pip', 'install', 'tornado==5.1.0'])
        

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

import warnings
warnings.filterwarnings("ignore")
import ipywidgets as widgets
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from tsmoothie.smoother import LowessSmoother
import plotly.express as px
from datetime import date
import ee
import geemap

try:
        from google.colab import files
        from google.colab import drive
        IN_COLAB = True
except:
        IN_COLAB = False



In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [37]:
#@title Run this cell to start or Reset the selection
# Functions

def smooth(serie):
    serieResult = pd.Series(serie.values)
    serieindex = pd.Series(serie.index)
    smoother = LowessSmoother(smooth_fraction=0.1, iterations=2)

    dataSmooth = smoother.smooth(serieResult.values)
    return pd.Series(dataSmooth.smooth_data[0],index=serieindex)
    
def hampel_filter_pandas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, center=True).mean()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median) * -1 
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def hampel_filter_pandas_positivas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, win_type = 'gaussian', center=True).mean(std=1)
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median)
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def cloud (image): 
    image = ee.Image(image)
    clouds = ee.Image(ee.List(image.get('cloud_mask'))).select('probability')
    clean = clouds.lte(80)
    qa = image.select('QA60')
    scl = image.select('SCL').eq(4).Or(image.select('SCL').eq(5))
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    finalmask = mask.And(clean)
    return image.updateMask(finalmask)

imageCollection = ee.ImageCollection("COPERNICUS/S2")
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask') \
                                    .apply(**{'primary': imageCollection,'secondary': s2Clouds, \
                                    'condition': ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})})



# Aggregation
def aggreg (image_collection, startdate : str ,enddate : str ,timeunit : str)-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string 
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    img_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(img_list).map(check, True)

# Aggregation
def aggreg_sum (image_collection, startdate : str ,enddate : str ,timeunit : str)-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string 
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.sum()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    imag_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(imag_list).map(check, True)

def SM_data (table):
    images = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture') \
    .filterDate('2015-06-01',str(date.today())) \
    .filterBounds(table) \
    .select(['susm'])

    start = ee.Date('2015-06-01')
    end = ee.Date('2022-06-30')
    timeunit = 'month'
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)

    def filter (date):
        images_g = ee.ImageCollection(images.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).multiply(1).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))

    img_list = ee.List.sequence(0, timedelta.int(),aggregation).map(step).map(filter)

    # Check missing values
    def check(image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image

    return ee.ImageCollection(img_list).map(check, True)

def GPM_data (table):
    images = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
    .filterDate('2022-01-01',str(date.today())) \
    .filterBounds(table) \
    .select(['precipitationCal'])

    start = ee.Date('2015-06-01')
    end = ee.Date('2022-06-30')
    timeunit = 'month'
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(images.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.sum()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    img_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(img_list).map(check, True)

def kendall_calc(collection):
  afterFilter = ee.Filter.lessThan(**{'leftField': 'system:time_start','rightField': 'system:time_start'})
  joined = ee.ImageCollection(ee.Join.saveAll('after').apply(**{'primary': collection,'secondary': collection,'condition': afterFilter}))

  def sign(i, j):
    return ee.Image(j).neq(i) \
            .multiply(ee.Image(j).subtract(i).clamp(-1, 1)).int()

  def func_oca(current):
    afterCollection = ee.ImageCollection.fromImages(current.get('after'))
    return afterCollection.map(lambda x : ee.Image(sign(current, x)).unmask(0))

  return ee.ImageCollection(joined.map(func_oca).flatten()).reduce('sum', 2)

gdf = gpd.read_file('https://raw.githubusercontent.com/Fernigithub/files/main/Frog_selsey_outwoods.geojson')
gdf.crs = "epsg:4326"
table_json = gdf.__geo_interface__
table = ee.FeatureCollection(table_json['features'])
coll = SM_data(table)
# kendall = kendall_calc(coll).resample('bicubic').reproject('EPSG:4326',None,5000)
kendall = kendall_calc(coll)
# minmax = kendall.reduceRegion(ee.Reducer.minMax(),table ,1000).getInfo()
# print(minmax)
GPM_coll = GPM_data(coll)
palette = ['red', 'white', 'green']

selected_fields = ee.FeatureCollection([])
Map = geemap.Map()
farms = ee.ImageCollection("projects/ee-fernandotentor/assets/Agreed/Farms")
Map.add_basemap('SATELLITE')
Map.addLayer(kendall,{'min':-150,'max':150,'palette':palette}, 'Soil M. Trend',False)
Map.addLayer(GPM_coll,{'min':0,'max':150,'palette':palette}, 'GPM_coll',False)
Map.addLayer(farms, {}, 'Farms_names',0)
Map.addLayer(table, {}, 'Farm')
Map.centerObject(table, 10)
Map.add_labels(table,'Field',font_color='yellow',font_size='10',draggable=False)
display(Map)

farms = gdf.Farm.unique()
n = 6 #items by line
checkbox_list = []
for farm in farms:
  data = gdf.loc[gdf['Farm'] == farm].Field
  checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]
  checkbox_list.append(checkboxes)

for j,check_group in enumerate(checkbox_list):
  i=0
  print(farms[j])
  for item in range(0,len(check_group),n ):
    output = widgets.HBox(children=check_group[i:i+n])
    i += n
    display(output)


Map(center=[51.24673120991562, -0.9381188984443471], controls=(WidgetControl(options=['position', 'transparent…

Frog


Outwoods


Selsey


In [38]:
#@title Run this cell to select the above checked fields
selected_data = []
selected = []
for j,check_group in enumerate(checkbox_list):
  for i in range(0, len(check_group)):
      if check_group[i].value == True:
          selected_data = selected_data + [check_group[i].description]

print(selected_data)
selected = table.filter(ee.Filter.inList("Field",selected_data))
selected_fields = selected_fields.merge(selected).distinct('Field')
Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
print('selected fields',len(selected_fields.getInfo()['features']))

['Blacksmiths', 'Near_Field', 'Intake', 'Brook_Field', 'Grain_Store', 'Drain_Field', '13_Acre', 'Slip', '9_Acre_Parsonage', 'Drain_Field_2', 'Barn_Field', 'Barn_Field_2', 'Far_Field', 'Leg_Field', 'Brook_Field_2', 'Bungalow_Field', 'Wood_Field', 'Roy_Jordans', 'Golf_course', '17_Acre', 'Glebe', '20_Acre', '15_Acre_MW', 'Rifle_Range', '12_Acre', 'Daniels', 'Over_MW']
selected fields 27


# Select fields GUI tool

In [39]:
#@title Run this cell to select the previously selected field by GUI 
roi = ee.FeatureCollection(Map.draw_features)
selected = table.filterBounds(roi.union(10))
print(selected.size().getInfo(),'features selected by drawing tool')
selected_fields = selected_fields.merge(selected).distinct('Field')
Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
try:
  Map.remove_drawn_features()
except:
  None
print('Fields selected', selected_fields.aggregate_array('Field').getInfo())



0 features selected by drawing tool
Fields selected ['Blacksmiths', 'Near_Field', 'Intake', 'Brook_Field', 'Grain_Store', 'Drain_Field', '13_Acre', 'Slip', '9_Acre_Parsonage', 'Drain_Field_2', 'Barn_Field', 'Barn_Field_2', 'Far_Field', 'Leg_Field', 'Brook_Field_2', 'Bungalow_Field', 'Wood_Field', 'Roy_Jordans', 'Golf_course', '17_Acre', 'Glebe', '20_Acre', '15_Acre_MW', 'Rifle_Range', '12_Acre', 'Daniels', 'Over_MW']


# Add other geometries [Optional]

In [40]:
#@title Run this cell after drawing new geometries at GUI
featcoll = []
i = 0
for feat in Map.draw_features:
    i +=1
    feat = feat.set({'properties':{'Farm':'N/A','Field':str(i)}})
    featcoll.append(feat)
print(len(featcoll),'features added')
added_geom = ee.FeatureCollection(featcoll)
try:
  selected_fields = selected_fields.merge(added_geom).distinct('Field')
except:
  selected_fields = added_geom
  Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
print('Fields selected', selected_fields.aggregate_array('Field').getInfo())
Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
Map.draw_features.clear()

try:
  Map.add_labels(selected_fields,'Field',font_color='yellow',font_size='10')
  Map.remove_drawn_features()
except:
  None


0 features added
Fields selected ['Blacksmiths', 'Near_Field', 'Intake', 'Brook_Field', 'Grain_Store', 'Drain_Field', '13_Acre', 'Slip', '9_Acre_Parsonage', 'Drain_Field_2', 'Barn_Field', 'Barn_Field_2', 'Far_Field', 'Leg_Field', 'Brook_Field_2', 'Bungalow_Field', 'Wood_Field', 'Roy_Jordans', 'Golf_course', '17_Acre', 'Glebe', '20_Acre', '15_Acre_MW', 'Rifle_Range', '12_Acre', 'Daniels', 'Over_MW']


# Save selected geometries

In [41]:
#@title Run this cell to save the selected geometries as GeoJSON.
# Save geometries as GeoJson 
geojson_feat = selected_fields.getInfo()
with open('selected_features.geojson', 'w') as f:
    json.dump(geojson_feat, f)
if IN_COLAB:
    files.download('selected_features.geojson')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# NDMI

In [42]:
#@title Run this cell to create NDMI Chart
#### obtain NDMI data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    ndmi = image.normalizedDifference(['B8','B11']).rename('ndmi')
    data = ndmi.reduceRegions(selected_fields ,ee.Reducer.mean(),10).filterMetadata('mean','greater_than',0)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    ndmi_value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,ndmi_value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'ndmi_mean',ndmi_value,'date',dia_list,'crop_id',crop_id)

images = s2SrWithCloudMask.filterBounds(selected_fields.geometry(10)).map(cloud)
S2_list_clean = aggreg(images ,'2017-01-01',str(date.today()),'month') #'day' or 'month'
image_list = ee.List(S2_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(S2_list_clean).map(data_extr).getInfo()


x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    ndmi_mean = feat['properties']['ndmi_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = ndmi_mean



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel[column] = hampel_filter_pandas(df_int[column],2,0.5)
    df_hampel[column] = hampel_filter_pandas_positivas(df_hampel[column],2,2)
    df_hampel_smooth[column] = smooth(df_hampel[column])
    
df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

# Plot NDMI 
df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_ndmi = df_plot
fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns , range_y=[0,1],
                    color_discrete_sequence=px.colors.qualitative.Light24,  
                    labels={
                     "value": "Index value",
                     "index": "NDMI",
                     "variable": "Fields"
                 },
                title="NDMI by Fields")
fig.show(figsize=(20,10))


## Save data

In [ ]:
#@title Run this cell to save NDMI as .CSV file
df_ndmi.to_csv('NDMI_data.csv')
df_ndmi.to_excel('NDMI_data.xls')

# NDVI

In [ ]:
#@title Run this cell to create NDVI Chart
#### obtain NDVI data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
    data = ndvi.reduceRegions(selected_fields ,ee.Reducer.mean(),10).filterMetadata('mean','greater_than',0)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    ndvi_value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,ndvi_value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'ndvi_mean',ndvi_value,'date',dia_list,'crop_id',crop_id)

images = s2SrWithCloudMask.filterBounds(selected_fields.geometry(10)).map(cloud)
S2_list_clean = aggreg(images ,'2017-01-01',str(date.today()),'month') #'day' or 'month'
image_list = ee.List(S2_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(S2_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    ndvi_mean = feat['properties']['ndvi_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = ndvi_mean



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel[column] = hampel_filter_pandas(df_int[column],2,0.5)
    df_hampel[column] = hampel_filter_pandas_positivas(df_hampel[column],2,2)
    df_hampel_smooth[column] = smooth(df_hampel[column])
    
df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_ndvi = df_plot
# Plot NDVI
fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns , range_y=[0,1],
                    color_discrete_sequence=px.colors.qualitative.Light24,
                    labels={
                     "value": "Index value",
                     "index": "NDVI",
                     "variable": "Fields"
                 },
                title="NDVI by Fields")
fig.show(figsize=(20,10))

In [ ]:
#@title Run this cell to save NDVI data as .CSV
df_ndvi.to_csv('NDVI_data.csv')
df_ndvi.to_excel('NDVI_data.xls')

# NOx

In [ ]:
#@title Run this cell to create NOx Chart
#### obtain NOx data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    data = image.reduceRegions(selected_fields ,ee.Reducer.mean(),100)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'nox_mean',value,'date',dia_list,'crop_id',crop_id)

images = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').select(['NO2_column_number_density'],['NO2'])
NOx_list_clean = aggreg(images ,'2017-01-01',str(date.today()),'month') #'day' or 'month'
image_list = ee.List(NOx_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(NOx_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    nox_mean = feat['properties']['nox_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = nox_mean



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel_smooth[column] = smooth(df_int[column])

df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names

# Plot Nox 
fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns ,
                    color_discrete_sequence=px.colors.qualitative.Light24,
                    labels={
                     "value": "NOx value",
                     "index": "NOx",
                     "variable": "Fields"
                 },
                title="NOx by Fields")
fig.show(figsize=(20,10))

# Soil Moisture

In [ ]:
#@title Run this cell to create Soil Moisture Chart
#### obtain Soil Moisture data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    data = image.reduceRegions(selected_fields ,ee.Reducer.mean(),100)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'susm_mean',value,'date',dia_list,'crop_id',crop_id)

images = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').select(['susm'],['susm'])
images_list_clean = aggreg(images ,'2017-01-01',str(date.today()),'month') #'day' or 'month'
image_list = ee.List(images_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(images_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    susm_mean = feat['properties']['susm_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = susm_mean



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel_smooth[column] = smooth(df_int[column])

df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names

# Plot data
fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns ,
                    color_discrete_sequence=px.colors.qualitative.Light24,
                    labels={
                     "value": "susm value",
                     "index": "susm",
                     "variable": "Fields"
                 },
                title="Soil Moisture by Fields")
fig.show(figsize=(20,10))

# Precipitation

In [ ]:
#@title Run this cell to create Precipitation Chart
#### obtain Precipitation data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    data = image.reduceRegions(selected_fields ,ee.Reducer.mean(),1000)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'gpm_mean',value,'date',dia_list,'crop_id',crop_id)

images = ee.ImageCollection("NASA/GPM_L3/IMERG_V06").select(['precipitationCal'],['precipitationCal'])
images_list_clean = aggreg_sum(images ,'2017-01-01',str(date.today()),'month') #'day' or 'month'
image_list = ee.List(images_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(images_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    gpm_mean = feat['properties']['gpm_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = gpm_mean

df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel_smooth[column] = smooth(df_int[column])

df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names

# Plot precipitation data

fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns ,
                    color_discrete_sequence=px.colors.qualitative.Light24,
                    labels={
                     "value": "Precipitation value",
                     "index": "Precipitation",
                     "variable": "Fields"
                 },
                title="Precipitation by Fields")
fig.show(figsize=(20,10))